# Datacenter Compute Rack
In this tutorial we will calculate the yearly energy consumption of a 19in server rack. 

Configurations:
  * 16 2U servers with 230VAC input
  * Server power (maximum): 850W
  * PDU distribution resistance: 2.5mOhm
  * Three different versions of 80plus certified PSUs: Bronze, Gold and Titanium

The rack is operated as follows during a year:
  * 1 day shutdown for maintenance
  * 75% of active time operating at 100% power
  * 20% of active time operating at 50% power
  * 5% of active time operating at 5% of power

In [1]:
# This cell can be removed, it is only used for running the notebook during Sphinx documentation build.
import sys, os
if os.getcwd().replace('\\', '/').endswith("/docs/nb"):
    sys.path.insert(0, os.path.abspath(os.path.join("../../src")))

In [2]:
from sysloss.components import *
from sysloss.system import System
import pandas as pd

## System definition
The function *create_rack()* below is used to create a system with the PSU efficiency parameter as input.

The following load phases are defined as well:
  * "Service"
  * "Full load"
  * "Half load"
  * "Idle"

```{note}
sysLoss treats AC and DC voltages the same. This is valid when the rms AC voltage is used on single phase (power factor 1).
```

In [3]:
DAY_SECS = 24*60*60 # seconds in a day
rack_phases = {"Service": DAY_SECS, "Full load": 364*DAY_SECS*0.75, "Half load": 364*DAY_SECS*0.75, "Idle": 364*DAY_SECS*0.05}

def create_rack(psu_efficiency):
    sys = System("Compute rack", source=Source("230VAC", vo=230.0))
    sys.set_sys_phases(rack_phases)
    for i in range(16):
        idx = "[{}]".format(i+1)
        sys.add_comp("230VAC", comp=RLoss("PDU resistance"+idx, rs=2.5e-3))
        sys.add_comp("PDU resistance"+idx, comp=Converter("PSU"+idx, vo=12.0, eff=psu_efficiency))
        sys.set_comp_phases("PSU"+idx, ["Full load", "Half load", "Idle"])
        sys.add_comp("PSU"+idx, comp=PLoad("Blade"+idx, pwr=850.0))
        sys.set_comp_phases("Blade"+idx, {"Full load": 850.0, "Half load": 425.0, "Idle": 42.5})
    return sys

Define efficiency for the three different PSU ratings:

In [4]:
bronze_eff = {"vi": [230.0], "io":[3.55, 7.1, 14.2, 35.5, 71.0], "eff":[[.67, .79, .85, .88, .85]]}
gold_eff =  {"vi": [230.0], "io":[3.55, 7.1, 14.2, 35.5, 71.0], "eff":[[.79, .86, .90, .92, .89]]}
titanium_eff =  {"vi": [230.0], "io":[3.55, 7.1, 14.2, 35.5, 71.0], "eff":[[.84, .90, .94, .96, .91]]}

## Analysis
Analysis is straight forward - run *solve()* with each of the three PSU ratings.

```{tip}
Set the *energy* parameter in *.solve()* to True - the results table will then contain a new column with the 24h energy consumption.
```

In [5]:
raitings = {"Bronze": bronze_eff, "Gold": gold_eff, "Titanium": titanium_eff}

res = []
for r in raitings.keys():
    rack = create_rack(raitings[r])
    res += [rack.solve(tags={"Rating": r}, energy=True)]
df = pd.concat(res, ignore_index=True)
df.tail()

,Component,Type,Parent,Rating,Phase,Vin (V),Vout (V),Iin (A),Iout (A),Power (W),Loss (W),Efficiency (%),24h energy (Wh),Warnings
598,PDU resistance[1],SLOSS,230VAC,Titanium,Idle,230.0,229.99945,0.21998,0.21998,50.595359,0.000121,99.999761,39.101297,
599,PSU[1],CONVERTER,PDU resistance[1],Titanium,Idle,229.99945,12.0,0.21998,3.541667,50.595238,8.095238,84.0,39.101203,
600,Blade[1],LOAD,PSU[1],Titanium,Idle,12.0,0.0,3.541667,0.0,42.5,0.0,100.0,32.845011,
601,System total,,,Titanium,Idle,,,,,809.525745,129.525745,83.999799,625.620746,
602,System average,,,Titanium,,,,,,10664.913806,789.513963,93.210233,255957.931346,


Since we are interested in the yearly power consumption, a new column is created for this:

In [8]:
df["Annual power (kWh)"] = df["24h energy (Wh)"] * 365 / 1000
df[df.Component == "System average"][["Component", "Rating", "Power (W)", "Loss (W)", "Efficiency (%)", "Annual power (kWh)"]].style.hide(axis='index')

Component,Rating,Power (W),Loss (W),Efficiency (%),Annual power (kWh)
System average,Bronze,11492.974920,1617.575080,85.894183,100678.460297
System average,Gold,10978.275298,1102.875456,90.146603,96169.691606
System average,Titanium,10664.913806,789.513963,93.210233,93424.644941


The power savings from using a Titanium certified PSU over a Bronze certified PSU is 7263kWh per year. Is there an economic gain to use higher rated PSUs? That depends on the lifetime of the rack, the energy prices and the cost difference between e.g. a Bronze PSU and Titanium PSU.

## Summary
This tutorial demonstrates how system energy consumption can be analyzed with sysLoss by enabling the *energy* parameter in *solve()*.